In [104]:
####################################################
# wifi 출석 검증 이슈 - 20180116
# 1. 강의실 ap 데이터 불충분 
# 2. 프로그래밍 과목 출석 검증 안됨 
# 3. wifi가 꺼진 경우, 스캐닝이 안되는 경우 존재 -> gps 활용 
# 4. x표시가 수업이 없는 요일인지, 스캐닝이 안된 건지 구분 필요 
####################################################
import csv
import numpy as np
import pandas as pd
import sqlite3
import os
from datetime import datetime,date

# class of student
class student():
    def __init__(self, stuID, studentsMeta, classsSchedule):
        # find studnet data whose ID is stuID in Student Meta data
        stuMeta = [i for i in studentsMeta if i[0] == stuID][0]
        # set meta data about this studnet
        self.stuID = stuMeta[0]
        self.sex = stuMeta[1]
        self.highschool = stuMeta[2]
        self.total = stuMeta[3]
        self.enrolled = stuMeta[4]
        self.drop = stuMeta[5]
        self.classes = stuMeta[6].split('~')
        self.classesSchedule = classsSchedule  # all classes schedule

        # set default wifi log, classes information that student enrolled,
        # wifi logs in classtime, and attendance
        self.log = []
        self.StuClassInfo = []
        self.logsInClass = {}
        self.scannedAP = {}
        self.attendance = {}
    
    # function that gets the student total wifi scanning log
    def getStuLog(self, stuID):
        stupath = '/Users/gimduyeon/Dropbox (KAIST Dr.M)/KAIST Dr.M의 팀 폴더/공유/rawdata/'
        #stupath = '/Users/duyeonee/Dropbox (KAIST Dr.M)/KAIST Dr.M의 팀 폴더/공유/rawdata/'
        files = os.listdir(stupath + str(stuID))
        logs = []
        for i in files:
            conn = sqlite3.connect(stupath + str(stuID) + '/' + i)
            cur = conn.cursor()
            cur.execute('SELECT * FROM HARDWARETABLE')
            [logs.append(row) for row in cur.fetchall() if row[2] == 'WIFI1' or row[2] == 'WIFI2']
        self.log = logs

    # function that gets classes information that student enrolled
    def getStuClassInfo(self, classSchedule):
        stuclass = pd.DataFrame(columns=['class', 'major', 'part', 'loc', 't1', 't2', 'BSSID'])
        for c in range(len(self.classes)):
            stuclass = pd.concat([stuclass, classSchedule[classSchedule['class'] == self.classes[c]]],
                                 join='outer')

        self.StuClassInfo = stuclass

    # function that gets the wifi scanning logs, that timestamps are in class time
    def getLogsInClass(self):
        logsinClass = {i: [] for i in self.classes}
        data_log_format = '%Y.%m.%d_%H.%M.%S'  # timestamp format

        # date 라이브러리에서 날짜가 월요일 ~ 일요일 까지 1~7로 매핑되어있다.
        # 숫자로 매핑된 날짜정보를 문자로 수정하기 위한 num_to_day
        num_to_day = {1: 'Mo', 2: 'Tu', 3: 'We', 4: 'Th', 5: 'Fr', 6: 'Sa', 7: 'Su'}

        # 수업 시간 도중에 발생한 wifi scanning data filtering 방법
        # 1. 전체 wifi scanning log를 하나씩 확인하다.
        # 2. 해당 log가 학생이 수강 중인 어떤 수업의 날짜와 같고, 시간이 그 수업 시간에 포함될때 -> 해당 수업 중 발생한 wifi scanning log
        # 그 이외의 경우 해당 수업시간에 발생한 wifi scanning log로 판별하지 않음
        for i in self.log:
            timestamp = datetime.strptime(i[1], data_log_format)  # logs' string format convert to datetime format
            weekday = num_to_day[date.isoweekday(timestamp)]  # 로그가 무슨 요일인지 확인
            daytime = timestamp.time()  # 로그가 몇시 몇분인지 확인

            # 전체 수업 중 로그가 어떤 수업 중 발생한 것인지 판별
            for j in range(len(self.classes)):
                classt1 = self.StuClassInfo.iloc[j, 4]  # 수업 시간
                if weekday == classt1.split("~")[0].split(".")[0]:  # 로그가 해당 수업이 있는 요일에 발생한 경우
                    classtime = [datetime.strptime(_.split(".")[1], '%H:%M') for _ in classt1.split("~")]
                    if classtime[0].time().minute < 20:
                        classtime[0] = classtime[0].replace(hour=classtime[0].time().hour-1,minute=classtime[0].time().minute-20+60)
                        if daytime > classtime[0].time() and daytime < classtime[1].time():  # 로그가 해당 수업 시간 중 발생하였을 때 로그를 해당 수업에 저장
                            logsinClass[self.classes[j]].append(i)
                    else:
                        classtime[0] = classtime[0].replace(minute=classtime[0].time().minute-20)
                        if daytime > classtime[0].time() and daytime < classtime[1].time():  # 로그가 해당 수업 시간 중 발생하였을 때 로그를 해당 수업에 저장
                            logsinClass[self.classes[j]].append(i)

                classt2 = self.StuClassInfo.iloc[j, 5]
                if classt2 is not '':  # 수업이 일주일에 두번일 경우. 위와 같은 과정을 거침
                    if weekday == classt2.split("~")[0].split(".")[0]:
                        classtime = [datetime.strptime(_.split(".")[1], '%H:%M') for _ in classt2.split("~")]
                        if classtime[0].time().minute < 20:
                            classtime[0] = classtime[0].replace(hour=classtime[0].time().hour-1,minute=classtime[0].time().minute-20+60)
                            if daytime > classtime[0].time() and daytime < classtime[1].time():  # 로그가 해당 수업 시간 중 발생하였을 때 로그를 해당 수업에 저장
                                logsinClass[self.classes[j]].append(i)
                        else:
                            classtime[0] = classtime[0].replace(minute=classtime[0].time().minute-20)
                            if daytime > classtime[0].time() and daytime < classtime[1].time():  # 로그가 해당 수업 시간 중 발생하였을 때 로그를 해당 수업에 저장
                                logsinClass[self.classes[j]].append(i)

                        
        self.logsInClass = logsinClass
    
    # function that checks attendance
    # 해당 수업 때 잡힌 ap의 갯수를 카운트 해서 리턴한다.
    def checkattend(self):
        print(self.stuID)
        logAP = {i: {} for i in self.classes}
        for c in self.classes:
            # 수업 중 발생한 로그 중, unknown ssid는 제거하고 나머지 로그의 날짜, bssid, ssid를 가져온다
            logs = [[i[1].split("_")[0], i[4], i[5]] for i in self.logsInClass[c] if i[3] != '<unknown ssid>']
            logs_perday = {i[0]: [] for i in logs}  # 요일별로 데이터 묶기
            [logs_perday[j[0]].append([j[1], j[2]]) for j in logs]
            logAP[c] = logs_perday

        info = {i: self.StuClassInfo[self.StuClassInfo['class'] == i].BSSID.values[0] for i in
                self.classes}  # 수업 별 ap 리스트 정보
        countAP = {i: {j: 0 for j in logAP[i]} for i in self.classes}
        listAP = {i: {j: 0 for j in logAP[i]} for i in self.classes}

        for c in self.classes:
            for k in logAP[c]:
                if info[c] != None:  # 강의실 ap 정보가 없는 경우는 예외처리
                    #listAP[c][k] = set([m for m in logAP[c][k][0] if isinstance(m,str) and m[:-2] in [z[:-2] for z in info[c]]]) #tanimoto 
                    listAP[c][k] = set([i[0] for i in logAP[c][k] if i[0] != None and i[0][:-2] in [z[:-2] for z in info[c]]])        
                    if len(listAP[c][k]) != 0:
                        countAP[c][k] = "출석: "+str(len(set(listAP[c][k])))+"/"+str(len(info[c]))  # 스캔된 ap 갯수 카운트
                    else:
                        countAP[c][k] = "일치wifi없음: "+str(len(listAP[c][k]))+"/"+str(len(info[c]))
                else:
                    print("강의실 ap정보 없음 ")
                    print(info[c])
        self.logAP = logAP
        self.info = info 
        self.listAP = listAP
        listAP = pd.DataFrame(listAP).fillna('.')
        countAP = pd.DataFrame(countAP).fillna('.')
        num_to_day = {1: 'Mo', 2: 'Tu', 3: 'We', 4: 'Th', 5: 'Fr', 6: 'Sa', 7: 'Su'}
        for c in self.classes:
            classday = [i.split(".")[0] for i in self.StuClassInfo.ix[self.StuClassInfo['class']==c,4:6].values[0]]
            for t in countAP.loc[:,c].keys():
                if num_to_day[datetime.strptime(t,'%Y.%m.%d').isoweekday()] in classday and countAP.loc[t,c] == '.':
                    countAP.loc[t,c] = 'x'
    
        self.scannedAP = listAP
        self.attendance = countAP
    # function that executes other class methods
    
    def studentUpdate(self):
        self.getStuLog(self.stuID)
        self.getStuClassInfo(self.classesSchedule)
        self.getLogsInClass()
        self.checkattend()


In [105]:
##############################################
########   데이터 불러오기 및 출석 체크  ############
##############################################

if __name__ == '__main__':
    # open AP data csv file
    #path = '/Users/duyeonee/Google Drive File Stream/My Drive/gitlab/drm-project/data'
    path = '/Users/gimduyeon/Google Drive File Stream/My Drive/gitlab/drm-project/data'
    f = open(path+'/classroomap_drm.csv','r')

    # AP data preprocessing
    aplist = [r for r in csv.reader(f)]
    aplist = [[i[0],i[1][1:-1].replace('"','').split(",")] for i in aplist]

    # get the classes' schedule data from scheduledatebase db file
    conn = sqlite3.connect(path+'/SCHEDULEDATABASE.db')
    cur = conn.cursor()
    cur.execute('SELECT * FROM SCHEDULETABLE')
    classschedule = [list(row) for row in cur.fetchall()]

    # integrate AP data and class schedule data by pandas format
    for i in classschedule:
        [i.append(j[1]) for j in aplist if j[0] == i[3]]
    classschedule_pd = pd.DataFrame(classschedule,columns=['class','major','part','loc','t1','t2','BSSID'])

    # get the student metadata from metadatabase db file
    studentsmeta = []
    conn = sqlite3.connect(path+'/METADATABASE.db')
    cur = conn.cursor()
    cur.execute('SELECT * FROM METATABLE')
    rows = cur.fetchall()
    for row in rows:
        studentsmeta.append(row)

    # 출석 체크 실행
    # sample students list
    testStu = [24897713,25063234,29962358,32646509,40523189,42236646,43261317,
                53789312,57623951,58201289,63987688,87155919,93775784,95983626]
    
    students = {i:0 for i in testStu}
    writer = pd.ExcelWriter(path+'/attendanceResults/studentsAttendance.xlsx')
    workbook = writer.book
    formats = workbook.add_format()
    formats.set_align('center')
    formats.set_align('vcenter')
    
    
    for i in testStu:
        students[i] = student(i,studentsmeta,classschedule_pd)
        students[i].studentUpdate()
        students[i].attendance.to_excel(writer,str(students[i].stuID))
        students[i].scannedAP.to_excel(writer,str(students[i].stuID)+"'s APlist")
    writer.save()

24897713


/Users/gimduyeon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:144: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


25063234
29962358
32646509
40523189
42236646
43261317
53789312
57623951
58201289
63987688
87155919
93775784
95983626


In [92]:
for c in students[43261317].classes:
    for k in students[43261317].logAP[c]:
        for m in students[43261317].logAP[c][k]:
            print(m[0])
        print("")

24:de:c6:db:18:e0
24:de:c6:db:18:e1
24:de:c6:db:18:e0
24:de:c6:db:28:c1
00:08:9f:0b:fa:30
24:de:c6:db:30:80
24:de:c6:db:30:81
24:de:c6:db:33:a1
00:12:5f:07:0d:e4
00:12:5f:07:90:20
24:de:c6:db:18:e0
24:de:c6:db:18:e0
00:08:9f:0b:fa:30
00:12:5f:07:0d:e4
24:de:c6:db:30:81
24:de:c6:db:28:c0
24:de:c6:db:30:80
00:1a:1e:7a:85:e0
00:1a:1e:7a:85:e1
24:de:c6:db:17:a0
24:de:c6:db:28:c1
24:de:c6:db:18:e1
00:1a:1e:78:be:c0
24:de:c6:db:34:00
24:de:c6:db:34:01
24:de:c6:db:17:a1
00:1a:1e:78:be:c1
24:de:c6:db:2f:80
2a:56:5a:21:a1:cb
24:de:c6:db:28:60
d8:c7:c8:dc:3e:80
24:de:c6:db:28:61
d8:c7:c8:dc:3e:81
24:de:c6:db:1b:00
5c:f3:70:10:3a:05
d8:c7:c8:df:24:20
00:12:5f:07:90:20
f4:d9:fb:d0:1a:af
f4:d9:fb:d0:1a:af
f4:d9:fb:cf:b3:ef
f4:d9:fb:d0:0d:af
f4:d9:fb:cf:ed:af
f4:d9:fb:cf:d7:af
f4:d9:fb:d0:1e:6f
f4:d9:fb:d0:1e:2f
f4:d9:fb:cf:df:ef
f4:d9:fb:d0:26:ef
f4:d9:fb:d0:1a:ef
f4:d9:fb:cf:a1:af

9c:1c:12:b8:c2:60
9c:1c:12:b8:c2:60
9c:1c:12:b8:c5:e0
32:cd:a7:fb:23:58
9c:1c:12:b8:be:e0
88:36:6c:23:0f:4a
90:9f:33:

30:89:d3:7e:8e:60
50:01:d9:cb:fb:22
d6:37:d7:46:6d:91
dc:d9:16:40:76:6d
dc:d9:16:40:76:6d
98:f1:99:9e:83:0e
9a:f1:99:9e:83:0e
44:c3:46:ac:6a:bd
70:8a:09:cd:c2:9a
80:13:82:07:86:55
60:83:34:e3:ce:50
ec:cb:30:c5:75:b3
c2:25:a2:12:1c:0f
d6:37:d7:46:6d:91
f6:dd:9e:93:0a:c9
14:9d:09:37:1d:61
30:89:d3:a7:8c:f0
dc:d9:16:40:76:6d
dc:d9:16:40:76:6d
98:f1:99:9e:83:0e
9a:f1:99:9e:83:0e
ec:cb:30:c5:75:b3
44:c3:46:ac:6a:bd
70:8a:09:cd:c2:9a
60:83:34:e3:ce:50
c2:25:a2:12:1c:0f
f6:a3:3d:aa:b6:a6
30:89:d3:7e:8e:60
c4:05:28:76:92:17
36:69:87:f5:d3:9a
dc:d9:16:40:76:6d
dc:d9:16:40:76:6d
dc:09:4c:0c:98:e4
70:8a:09:cd:c2:9a
0c:8f:ff:fe:82:58
74:a5:28:78:78:25
30:89:d3:63:94:70
9a:f1:99:ad:ae:1e
9c:1c:12:d2:f0:00
9c:1c:12:d2:f0:02
9c:1c:12:d2:f0:01
9c:1c:12:e2:4a:82
9c:1c:12:e2:4a:80
98:f1:99:ad:ae:1e
36:de:34:cc:b6:fa
dc:d9:16:40:76:6d
dc:d9:16:40:76:6d
e0:a3:ac:b4:ba:3c
48:db:50:b1:4d:ff
8c:eb:c6:9a:d8:02
b4:30:52:bc:ec:01
48:db:50:b1:4c:13
30:89:d3:5c:ae:e0
d4:a1:48:50:79:a1
c2:25:a2:a4:64:13
30:89:d3:4

f4:d9:fb:d0:0d:af
f4:d9:fb:d0:1a:ef
f4:d9:fb:d0:1d:af
f4:d9:fb:d0:1a:af
f4:d9:fb:cf:a1:af
f4:d9:fb:cf:de:2f
f4:d9:fb:cf:ed:af
f4:d9:fb:cf:a1:ef
f4:d9:fb:cf:d5:6f
f4:d9:fb:cf:b3:ef
f4:d9:fb:cf:b3:ef
f4:d9:fb:d0:0d:af
f4:d9:fb:d0:1a:ef
f4:d9:fb:d0:1d:af
f4:d9:fb:d0:1a:af
f4:d9:fb:cf:a1:af
f4:d9:fb:cf:de:2f
f4:d9:fb:cf:ed:af
f4:d9:fb:cf:a1:ef
f4:d9:fb:cf:df:ef
f4:d9:fb:d0:1e:6f
f4:d9:fb:d0:1e:2f
f4:d9:fb:cf:d7:af
f4:d9:fb:cf:b3:ef
f4:d9:fb:cf:b3:ef
f4:d9:fb:d0:1a:af
f4:d9:fb:cf:a1:af
f4:d9:fb:cf:de:2f
f4:d9:fb:d0:1e:6f
f4:d9:fb:d0:1a:ef
f4:d9:fb:cf:df:ef
f4:d9:fb:d0:0d:af
f4:d9:fb:cf:ed:af
f4:d9:fb:d0:1d:af
f4:d9:fb:cf:f3:2f
f4:d9:fb:cf:d7:af
f4:d9:fb:d0:26:ef
f4:d9:fb:d0:2a:2f
f4:d9:fb:cf:f5:6f

d8:c7:c8:de:f8:01
d8:c7:c8:de:f8:01
d8:c7:c8:de:f8:00
2a:56:5a:21:83:5d
00:1a:1e:7a:8c:01
00:1a:1e:78:be:c1
00:1a:1e:7a:8c:00
24:de:c6:e3:2f:c1
24:de:c6:e3:2f:c0
d8:c7:c8:df:2c:e1
00:1a:1e:7a:8d:81
d8:c7:c8:df:19:e1
d8:c7:c8:df:2d:80
d8:c7:c8:de:41:00
d8:c7:c8:de:41:01
2a:56:5a:21:a1:cb
d8:c7:c8:

f4:d9:fb:cf:d7:af
f4:d9:fb:d0:1a:af
f4:d9:fb:cf:ed:af
f4:d9:fb:d0:1e:6f
f4:d9:fb:d0:1d:af
f4:d9:fb:cf:f3:2f
f4:d9:fb:cf:b3:ef
f4:d9:fb:cf:b3:ef
f4:d9:fb:d0:1a:af
f4:d9:fb:cf:ed:af
f4:d9:fb:d0:1e:6f
f4:d9:fb:cf:de:2f
f4:d9:fb:d0:0d:af
f4:d9:fb:d0:1d:af
f4:d9:fb:cf:a1:af
f4:d9:fb:d0:26:ef
f4:d9:fb:d0:1a:ef
f4:d9:fb:d0:1e:2f
f4:d9:fb:cf:a1:ef
f4:d9:fb:cf:df:ef

d8:c7:c8:df:22:81
d8:c7:c8:df:22:81
d8:c7:c8:de:f5:a1
d8:c7:c8:df:23:e1
d8:c7:c8:df:0f:01
d8:c7:c8:df:22:41
d8:c7:c8:df:22:41
00:26:66:81:9e:08
64:e5:99:7e:19:ba
d8:c7:c8:df:22:01
00:24:6c:5b:87:40
90:9f:33:db:a8:24
24:de:c6:db:17:a0
2a:56:5a:61:b7:28
24:de:c6:db:1b:01
24:de:c6:db:1b:00
24:de:c6:db:17:a1
24:de:c6:d9:f1:81
24:de:c6:db:17:a0
24:de:c6:d9:f1:80
d8:c7:c8:dc:3e:80
24:de:c6:db:28:c1
24:de:c6:db:28:c0
d8:c7:c8:df:19:e0
d8:c7:c8:df:19:e1
d8:c7:c8:dc:3e:81
2c:30:33:f3:7e:12
00:24:6c:5b:87:41
24:de:c6:db:28:c0
24:de:c6:db:28:c0
24:de:c6:db:28:c1
00:08:9f:0b:fa:30
00:24:6c:5b:87:40
00:24:6c:5b:87:41
00:12:5f:07:90:20
24:de:c6:

00:1a:1e:7a:8d:a1
d8:c7:c8:df:2d:81
d0:95:c7:41:4f:bf
00:1a:1e:7a:8d:a0
24:de:c6:e3:2c:81
b4:5d:50:a7:58:80
d8:c7:c8:df:2d:80
d8:c7:c8:dc:3f:80
d8:c7:c8:df:24:20
18:64:72:0e:b0:80
8e:72:38:d3:39:61

00:1a:1e:7a:8d:c0
00:1a:1e:7a:8d:c0
d8:c7:c8:de:f8:a1
d8:c7:c8:de:f8:a1
24:de:c6:e3:2c:80
24:de:c6:e3:2c:81
00:1a:1e:75:ef:81
64:e5:99:9c:36:1a
d8:c7:c8:df:2d:81
de:0b:34:b6:97:c9
00:1a:1e:7a:8c:c1
00:1a:1e:7a:8c:c0
d8:c7:c8:df:2c:e1
d8:c7:c8:df:2c:e0
00:1a:1e:75:ef:80
24:de:c6:e3:2f:c1
00:1a:1e:7a:8d:a1
24:de:c6:e3:2f:c0
00:1a:1e:78:be:c1
d8:c7:c8:df:2d:80
00:1a:1e:7a:8d:a0
18:64:72:0e:b0:60
d8:c7:c8:de:f8:01
d8:c7:c8:df:22:c0
d8:c7:c8:df:22:c1
d8:c7:c8:de:f8:a1
d8:c7:c8:de:f8:a1
24:de:c6:e3:2c:80
24:de:c6:e3:2c:81
d8:c7:c8:de:f8:a0
d8:c7:c8:dc:3f:81
d8:c7:c8:dc:3f:80
00:1a:1e:7a:8c:c1
00:1a:1e:7a:8c:c0
d8:c7:c8:df:2c:e0
d8:c7:c8:df:2c:e1
00:1a:1e:75:ef:81
00:1a:1e:75:ef:80
64:e5:99:6b:ac:dc
64:e5:99:9c:36:1a
24:de:c6:e3:2f:c0
24:de:c6:e3:2f:c1
d8:c7:c8:df:2d:80
d8:c7:c8:df:2d:81
00:1a:1e:

90:9f:33:ac:7c:32
90:9f:33:de:56:a4
24:de:c6:e3:43:e0
00:26:66:f6:b4:1c
24:de:c6:ec:65:a0
64:e5:99:7f:0d:8a
90:9f:33:98:47:c6
90:9f:33:66:38:6c
d8:c7:c8:de:3a:a1
d8:c7:c8:df:17:a1
d8:c7:c8:df:02:61
00:26:66:86:9b:f8
24:de:c6:e3:43:e0
90:9f:33:de:56:a4
24:de:c6:e3:43:e0
24:de:c6:ec:65:a0
d8:c7:c8:de:fb:21
24:de:c6:db:1b:e0
d8:c7:c8:de:3a:a1
00:1a:1e:75:f1:80
00:1a:1e:7a:8c:00
00:1a:1e:7a:8c:01
00:1a:1e:75:f1:80
58:1d:91:b9:1f:88
24:de:c6:e3:2b:20
00:1a:1e:75:f1:81
24:de:c6:e3:2b:21
00:1a:1e:78:c2:00
00:1a:1e:78:c2:01
24:de:c6:e3:32:01
24:de:c6:e3:32:00
00:1a:1e:7a:7c:c1
00:24:6c:5b:90:40
00:0e:8e:67:31:01
00:1a:1e:75:f1:80
00:1a:1e:7a:8c:00
00:1a:1e:75:f1:80
00:1a:1e:7a:8c:01
58:1d:91:b9:1f:88
24:de:c6:e3:2b:21
90:9f:33:bb:87:60
2a:56:5a:61:d9:2a
d8:c7:c8:df:13:a1

f4:d9:fb:cf:b3:ef
f4:d9:fb:cf:b3:ef
f4:d9:fb:d0:0d:af
f4:d9:fb:cf:de:2f
f4:d9:fb:d0:1a:af
f4:d9:fb:d0:1e:6f
f4:d9:fb:d0:1d:af
f4:d9:fb:cf:ed:af
f4:d9:fb:d0:1a:ef
f4:d9:fb:cf:b3:ef
f4:d9:fb:cf:b3:ef
f4:d9:fb:d0:0d:af
f4:d9:fb:

In [73]:
s = [['a','b','c','d']]
t = ['c','d']
[i[0] for i in s if i[0] in t] 

[]

In [87]:
students[43261317].logAP['AdvancedEnglishWriting-G']['2017.09.06'][13][0] != None

False